In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
# entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [4]:
from ML_PROJECT.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from ML_PROJECT.utils.common import create_directories, read_yaml



class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [5]:
from ML_PROJECT import logger
from sklearn.model_selection import train_test_split
import pandas as pd


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)

        train, test = train_test_split(data)
        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index=False)

        logger.info("Splitted data into training amnd test sets")
        logger.info(train.shape)
        logger.info(test.shape)

In [6]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2025-05-24 13:15:28,294: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-24 13:15:28,295: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-24 13:15:28,296: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-24 13:15:28,296: INFO: common: created directory at: artifacts]
[2025-05-24 13:15:28,296: INFO: common: created directory at: artifacts/data_transformation]
[2025-05-24 13:15:28,306: INFO: 2158817557: Splitted data into training amnd test sets]
[2025-05-24 13:15:28,307: INFO: 2158817557: (1499, 12)]
[2025-05-24 13:15:28,307: INFO: 2158817557: (500, 12)]
